In [1]:
pip install -r requirements-no-cartopy.txt

  Using cached h5py-2.10.0-cp37-cp37m-win_amd64.whl (2.5 MB)
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import netCDF4
nc = netCDF4.Dataset("data/raw/merged.nc")
X_train_nc = nc.variables["precip"][:]

import numpy as np
import pandas as pd
X_train = X_train_nc.data
X_train[X_train==-9999] = np.nan

In [3]:
X_train.shape

(15340, 540, 310)

In [ ]:
(pd.DataFrame(nc.variables["lat"][:].data)).to_csv("lat.csv")
(pd.DataFrame(nc.variables["lon"][:].data)).to_csv("lon.csv")

In [27]:
nc.variables["lat"][:].data.shape

(540,)

In [ ]:
# ############IMPORT##########################
import tensorflow as tf
from keras.backend import set_session
from src.modeling import wgan_gp_V8_82c as wg
from src.preparation import data_preproc as preproc

# Setting for memory allocaton of the GPU.
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
sess = tf.Session()

In [61]:
wgan = wg.WGANGP(latent_dim=64, target_shape=(540, 310, 1), batch_size=2,
                 optimizerG=None, optimizerC=None, summary=True, n_critic=1,
                 models=None, gradient_penalty=10, data=X_train_nc[:10000, :, :],
                 tfboard=False)

wgan.train(epochs=5, save_interval=4, save_file='test',
           run_name='run_test', log_interval=10, log_file='test',
           data_generator=None, save_intermediate_model=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_106 (InputLayer)          (None, 540, 310, 1)  0                                            
__________________________________________________________________________________________________
wrap_padding2d_131 (WrapPadding (None, 540, 320, 1)  0           input_106[0][0]                  
__________________________________________________________________________________________________
conv2d_532 (Conv2D)             (None, 538, 318, 64) 640         wrap_padding2d_131[0][0]         
__________________________________________________________________________________________________
leaky_re_lu_26 (LeakyReLU)      multiple             0           conv2d_532[0][0]                 
                                                                 conv2d_533[0][0]                 
          

ValueError: Operands could not be broadcast together with shapes (540, 310, 1) (64, 128, 1)

In [5]:
lat = np.genfromtxt("./data/raw/lat.csv", delimiter=",")  # shape:(64)
llat = np.expand_dims(lat, axis=0)  # shape : (1, 64)
llat = np.expand_dims(llat, axis=-1)  # shape : (1, 64, 1)
llat = np.expand_dims(llat, axis=-1)  # shape : (1, 64, 1, 1)
# llat = llat.repeat(N_train, axis = 0)
llat = llat.repeat(310, axis=2)  # shape : (1, 64, 128, 1)
llat.shape

(1, 540, 310, 1)